In [99]:
# imports
import pandas as pd
import os # use this to access your environment variables
import requests # this will be used to call the APIs
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import json

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [136]:
#define function for getting POIs etc. from foursquare
def get_venues_fs(latitude, longitude, radius, api_key, categories):
    url_fs = "https://api.foursquare.com/v3/places/search"
    

    params = {
        "ll": f"{latitude},{longitude}",
        "radius": radius,
        "categories": categories,
        "fields": "fsq_id,name,geocodes,location,categories,distance,rating,price,closed_bucket"
    }

    headers = {
        "Accept": "application/json",
        "Authorization": api_key
    }


    # """
    # Get venues from foursquare with a specified place type and coordinates.
    # Args:
    #     latitude (float): latitude for query (must be combined with longitude)
    #     longitude (float): longitude for query (must be combined with latitude)
    #     api_key (str): foursquare API to use for query
    #     categories (str) : Foursquare-recognized place type. If not passed no place_type will be specified. 
    #     Separate ids with commas
    
    # Returns:
    #     response: response object from the requests library.
    # """
    response = requests.request("GET", url_fs, headers=headers, params=params)

    return response

Iterate through all locations

In [82]:
# load bike stations .csv
tor_csv = pd.read_csv('tor_stations.csv')
tor_csv.info()

tor_csv.sample(n=200).reset_index(drop=True).to_csv('tor_stations_sample.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 751 entries, 0 to 750
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   empty_slots  751 non-null    int64  
 1   free_bikes   751 non-null    int64  
 2   id           751 non-null    object 
 3   latitude     751 non-null    float64
 4   longitude    751 non-null    float64
 5   name         751 non-null    object 
 6   timestamp    751 non-null    object 
dtypes: float64(2), int64(2), object(3)
memory usage: 41.2+ KB


In [96]:
#assign api key variable
fs_key = os.getenv('FOURSQUARE_API_KEY')
radius = 1000
categories = '13003, 12080, 16032, 13065, 19042, 19043, 19046, 19054, 19067'

In [101]:
fs_key

'fsq3E6A3NqmjGtMTfxyXK6KU0IRj74JX1NakuK7+E49mCcg='

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [84]:
#load sample of 200 stations
tor_csv_200 = pd.read_csv('tor_stations_sample.csv')
tor_csv_200

,empty_slots,free_bikes,id,latitude,longitude,name,timestamp
0,3,23,dcbcf9c681fbd3e77dfbea6df0e070bc,43.645857,-79.385365,Simcoe St / Wellington St W South,2023-11-17 15:55:45.502000-05:00
1,0,18,d1de2ddc0ea8a90d00ff717ad60a2f23,43.638497,-79.391920,Lower Spadina Ave / Lake Shore Blvd W,2023-11-17 15:55:44.622000-05:00
2,3,12,9fd381407b51a700753caec17607a127,43.647508,-79.386044,Simcoe St / King St W,2023-11-17 15:55:45.336000-05:00
3,3,16,f42363acf4af847ca6fcdbe9b48c8290,43.637981,-79.424146,Liberty St / Fraser Ave Green P,2023-11-17 15:55:45.472000-05:00
4,14,2,c4a01864ecb66e9b1ee3023d3205a281,43.680022,-79.340568,Langford Ave / Danforth Ave - SMART,2023-11-17 15:55:45.220000-05:00
...,...,...,...,...,...,...,...
195,13,0,f36d8ed5ff407fc4eab6c9cc344661f9,43.687323,-79.304848,Danforth Ave / Westlake Ave,2023-11-17 15:55:45.593000-05:00
196,8,6,719e676d739529c8d5da8bb89c5c8678,43.695019,-79.271807,Danforth Ave / Warden Ave,2023-11-17 15:55:45.555000-05:00
197,0,23,d0a8478c786cb2480de4fcecda844781,43.643975,-79.419576,Queen St W / Ossington Ave,2023-11-17 15:55:44.670000-05:00
198,19,1,f0af815a9e6c12bd0382843e03fd322f,43.706347,-79.401610,Eglinton Ave W / Henning Ave SMART,2023-11-17 15:55:45.562000-05:00


In [140]:

#iterate through each row and retrieve latitude and longitude, passing them through get_venues_fs
pois = []

for index, row in tor_csv_200.iterrows():
    stations = get_venues_fs(tor_csv_200.at[index, 'latitude'], tor_csv_200.at[index, 'longitude'], radius=radius, api_key=fs_key, categories=categories)
    stations_json = stations.json()
    for item in stations_json['results']:
    # Extract the values for each desired key, with checks for existence of keys
        names = []
        [names.append(cat['name']) for cat in item['categories']]
        pois.append({
            'closest bike station id': tor_csv_200.loc[index, 'id'],
            'poi_id': item.get('fsq_id'),
            'poi_name': item.get('name'),
            'address': item['location']['formatted_address'] if item.get('location') else None,
            'distance': item.get('distance'),
            'longitude': item['geocodes']['main']['longitude'] if item.get('geocodes') and item['geocodes'].get('main') else None,
            'latitude': item['geocodes']['main']['latitude'] if item.get('geocodes') and item['geocodes'].get('main') else None,
            'categories': names,
            'rating': item.get('rating'),
            'price': item.get('price')
        })

Put your parsed results into a DataFrame

In [141]:
fs_df = pd.DataFrame(pois)

fs_df

,closest bike station id,poi_id,poi_name,address,distance,longitude,latitude,categories,rating,price
0,dcbcf9c681fbd3e77dfbea6df0e070bc,529612de11d2ab526191ccc9,Pai,"18 Duncan St (at Adelaide St W), Toronto ON M5...",345,-79.388636,43.647796,[Thai Restaurant],9.3,2.0
1,dcbcf9c681fbd3e77dfbea6df0e070bc,5462ac56498e128ccafe8fea,Pizzeria Libretto,"155 University Ave (at Pearl St), Toronto ON M...",275,-79.384955,43.648456,[Pizzeria],8.7,2.0
2,dcbcf9c681fbd3e77dfbea6df0e070bc,5c08474447f876002c736b68,Wvrst Union Station,"65 Front St W, Toronto ON M5J 1E6",335,-79.381397,43.644898,[Beer Bar],8.9,NaN
3,dcbcf9c681fbd3e77dfbea6df0e070bc,50322b6ae4b09116a296568c,Soho House,"192 Adelaide St W (at Simcoe St.), Toronto ON ...",333,-79.386412,43.648800,"[Speakeasy, Social Club]",8.9,2.0
4,dcbcf9c681fbd3e77dfbea6df0e070bc,5b6c842bc36588002c80a934,Cafe Landwer,"165 University Ave (Adelaide St W), Toronto ON...",321,-79.385137,43.648724,"[Café, Restaurant]",8.8,1.0
...,...,...,...,...,...,...,...,...,...,...
1986,0abdfc3177000757dcd71bd4c8b479ee,4af5a445f964a520b4fa21e3,Vermont Square Park,"819 Palmerston Ave (Bathurst & Dupont area), T...",440,-79.415980,43.670004,[Playground],7.9,NaN
1987,0abdfc3177000757dcd71bd4c8b479ee,54713d37498ec7371141032e,Civil Liberties,"878 Bloor St W (at Ossington Ave), Toronto ON ...",945,-79.425123,43.662264,[Cocktail Bar],8.8,3.0
1988,0abdfc3177000757dcd71bd4c8b479ee,50456a23e4b0a30df06f1391,Tallboys Craft Beer House,"838 Bloor St W (at Shaw St.), Toronto ON M6G 1M2",869,-79.423902,43.662521,"[Pub, Restaurant]",8.6,2.0
1989,0abdfc3177000757dcd71bd4c8b479ee,54400bee498e61defb21f0a5,Schmaltz Appetizing,"414 Dupont St, Toronto ON M5R 1V9",844,-79.411685,43.673854,[Deli],8.5,1.0


In [142]:
#save df to csv
fs_df.to_csv('fs_df.csv', index=False)

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [60]:
#yelp endpoint
yelp_url = 'https://api.yelp.com/v3'

#business search endpoint
ylp_endpoint = '/businesses/search'

#yelp api key
yelp_key = os.getenv('YELP_API_KEY')

In [59]:
#create function for calling yelp api
def get_venues_yelp(latitude, longitude,  api_key, radius=1000, categories=None, endpoint=None):
    # Get venues from foursquare with a specified place type and coordinates.
    # Args:
    #     latitude (float): latitude for query
    #     longitude (float): longitude for query
    #     api_key (str): yelp API to use for query
    #     categories (str) : yelp-recognized place type. If not passed no place_type will be specified. 
    # Returns:
    #     response: response object from the requests library.
    
    if endpoint is not None:
        url = 'https://api.yelp.com/v3' + endpoint
    else:
        url = 'https://api.yelp.com/v3/'

    params = {
        "latitude": latitude,
        "longitude": longitude,
        "radius": radius,
        "categories": categories,
        "limit": 10
    }

    headers = {
        "Accept": "application/json",
        'Authorization': 'Bearer %s' % api_key
    }

    response = requests.request("GET", url, headers=headers, params=params)

    return response

In [137]:
#testing calls with single location
#43.654905	-79.398448 - Spadina and Baldwin
testlat = 43.654905
testlon = -79.398448
rad = 1000
test1 = get_venues_fs(latitude=testlat, longitude=testlon, radius=rad, api_key=fs_key, categories='13003')

In [138]:
test1.json()

{'results': [{'fsq_id': '589bc46a739d8546efb5f2fd',
   'categories': [{'id': 13025,
     'name': 'Wine Bar',
     'short_name': 'Wine Bar',
     'plural_name': 'Wine Bars',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/winery_',
      'suffix': '.png'}},
    {'id': 13065,
     'name': 'Restaurant',
     'short_name': 'Restaurant',
     'plural_name': 'Restaurants',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
      'suffix': '.png'}}],
   'closed_bucket': 'VeryLikelyOpen',
   'distance': 287,
   'geocodes': {'drop_off': {'latitude': 43.653759, 'longitude': -79.401708},
    'main': {'latitude': 43.653818, 'longitude': -79.401564},
    'roof': {'latitude': 43.653818, 'longitude': -79.401564}},
   'location': {'address': '199 Augusta Ave',
    'country': 'CA',
    'cross_street': '',
    'formatted_address': '199 Augusta Ave, Toronto ON M5T 2L4',
    'locality': 'Toronto',
    'postcode': 'M5T 2L4',
    'region': 'ON'},
   'name': 

In [85]:
ylptestjson = test1.json()

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
rad = 1000
categories = 'bars, restaurants, libraries, parks, cafes, metrostations, busstations'

In [155]:
#parse through sample of 200 stations
pois_ylp = []

for index, row in tor_csv_200.iterrows():
    stations_ylp = get_venues_yelp(tor_csv_200.at[index, 'latitude'], tor_csv_200.at[index, 'longitude'], radius=rad, api_key=yelp_key, categories=categories, endpoint=ylp_endpoint)
    stations_ylp_json = stations_ylp.json()
    for item in stations_ylp_json['businesses']:
    #extract values for each desired key 
        cat_names = []
        [cat_names.append(cat['title']) for cat in item['categories']]
        pois_ylp.append({
            'closest bike station id': tor_csv_200.loc[index, 'id'],
            'poi_id': item.get('id'),
            'name': item.get('name'),
            'address': item.get('location').get('display_address')[0] if item.get('location') else None,
            'distance': item.get('distance'),
            'longitude': item.get('coordinates').get('longitude') if item.get('coordinates') else None, 
            'latitude': item.get('coordinates').get('latitude') if item.get('coordinates') else None,
            'categories': cat_names,
            'rating': item.get('rating'),
            'price': item.get('price')
        })

Put your parsed results into a DataFrame

In [156]:
ylp_df = pd.DataFrame(pois_ylp)
ylp_df

,closest bike station id,poi_id,name,address,distance,longitude,latitude,categories,rating,price
0,dcbcf9c681fbd3e77dfbea6df0e070bc,r_BrIgzYcwo1NAuG9dLbpg,Pai Northern Thai Kitchen,18 Duncan Street,348.204192,-79.388720,43.647840,[Thai],4.5,$$
1,dcbcf9c681fbd3e77dfbea6df0e070bc,Yl2TN9c23ZGLUBSD9ks5Uw,Byblos,11 Duncan Street,290.701203,-79.388123,43.647546,"[Mediterranean, Middle Eastern]",4.5,$$$$
2,dcbcf9c681fbd3e77dfbea6df0e070bc,jGO4T6rGX3EIOwOtg4bZqg,Lobster Burger Bar,214 King Street W,209.383312,-79.386768,43.647443,"[Bars, Seafood, Burgers]",4.0,$$
3,dcbcf9c681fbd3e77dfbea6df0e070bc,ocLnCE2E29j-CoBQj1yaSA,Cafe Landwer - Adelaide & University,165 University Avenue,325.377024,-79.384910,43.648750,"[Cafes, Breakfast & Brunch, Mediterranean]",4.0,$$
4,dcbcf9c681fbd3e77dfbea6df0e070bc,JMiaNitMzMbJm6Kh0RbT5A,Canoe,66 Wellington Street W,285.381498,-79.382159,43.646954,[Canadian (New)],4.0,$$$$
...,...,...,...,...,...,...,...,...,...,...
1980,0abdfc3177000757dcd71bd4c8b479ee,-ltD-dHqdZ5-wjAMKYa6Ng,La Bella Managua,872 Bloor Street W,919.489403,-79.424950,43.662320,"[Latin American, Nicaraguan]",4.5,$$
1981,0abdfc3177000757dcd71bd4c8b479ee,QA30Lo1NlV8SjnkmEkd1-A,Daldongnae Korean BBQ - Christie,658 Bloor Street,758.561743,-79.415660,43.664340,"[Korean, Barbeque]",4.5,$$$
1982,0abdfc3177000757dcd71bd4c8b479ee,oZP0Ynj-yHUxJUmY7-HaUA,Rustle & Still Café,605 Bloor Street W,868.963300,-79.413040,43.664570,"[Coffee & Tea, Vietnamese]",4.5,$
1983,0abdfc3177000757dcd71bd4c8b479ee,nHFJtud7jWZhM9dHQ1eIRA,Buk Chang Dong Soon Tofu,691 Bloor Street W,784.093681,-79.416930,43.663630,[Korean],4.0,$$


In [157]:
ylp_df.to_csv('ylp_df.csv', index=False)

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Although Foursquare may have more data per business for a similar query, much of it seems redundant or unnecessary, such as the longitude and latitude listed multiple times. 

Yelp has the rating and a price rating for each establishment which is only two more data points but offers a large benefit over using foursquare. Foursquare also has this information but is behind a paywall, making it less accessible. 

After some manual inspectiong, however, it appears that yelp has POIs that have since permanently closed but are still returned in the request. There does not appear to be a parameter that can be passed to ensure that the business are still operating at the same address. 

Foursquare offers some insight into whether or not the establishment is still open with their 'closed_bucket' attribute. With the shortcomings of yelp with regards to businesses still in operation, Foursquare is the more complete data source. 

Get the top 10 restaurants according to their rating

As Yelp's rating system is on a 5 point scale with 0.5 resolution, and some POIs with 5 start ratings have been confirmed to be permanently closed, the Foursquare data will be used instead for the top 10 restaurants.

In [152]:
#Top 10 restaurants by rating
top_ten = fs_df.drop_duplicates(subset=['poi_name']).sort_values(by='rating', ascending=False)

In [153]:
top_ten[~top_ten['poi_name'].str.contains('park', case=False)].head(10)

,closest bike station id,poi_id,poi_name,address,distance,longitude,latitude,categories,rating,price
416,71debce521f35fd4e280e8ee1e2dd8cc,550dfcb6498e446e1eff9031,Northern Belle,"913 Dundas St W (at Bellwoods Ave), Toronto ON...",627,-79.412661,43.650838,"[Cocktail Bar, Café, Restaurant]",9.4,3.0
83,5539f3700e4ce9e5701353a667924538,4f7891c7e4b0b9643b73e08d,Bellwoods Brewery,"124 Ossington Ave (at Argyle St), Toronto ON M...",162,-79.420177,43.647131,"[Bar, Brewery, Restaurant]",9.3,3.0
0,dcbcf9c681fbd3e77dfbea6df0e070bc,529612de11d2ab526191ccc9,Pai,"18 Duncan St (at Adelaide St W), Toronto ON M5...",345,-79.388636,43.647796,[Thai Restaurant],9.3,2.0
256,7a666ee6c60a7f8ffec9334924d36c6a,5321f4d9e4b07946702e6e08,Byblos,"11 Duncan St (at Pearl St), Toronto ON M5V 3M2",690,-79.388264,43.647450,[Mediterranean Restaurant],9.3,2.0
6,dcbcf9c681fbd3e77dfbea6df0e070bc,4ad4c05ef964a520cbf620e3,Steam Whistle Brewing,"255 Bremner Blvd (btwn Rees St. & Simcoe St.),...",476,-79.385267,43.641025,"[Brewery, Restaurant]",9.2,1.0
87,5539f3700e4ce9e5701353a667924538,4af369d0f964a52060ed21e3,Foxley,"207 Ossington Ave (Dundas), Toronto ON M6J 2Z8",339,-79.420333,43.648691,"[Bistro, Chinese Restaurant, Thai Restaurant]",9.2,2.0
327,8db545a0135f7b92022fbdbbc75af6c4,527d450111d25050de4ea0d8,Rasa,"196 Robert St, Toronto ON M5S 2K7",754,-79.404126,43.662838,[Restaurant],9.2,NaN
407,cc1432c52b1e87a738180d369ed1136b,4ad4c062f964a520fbf720e3,St. Lawrence Market Outdoor Vendors,"93 Front St E (at Lower Jarvis St), Toronto ON...",736,-79.371614,43.648637,"[Restaurant, Farmers Market, Grocery Store]",9.2,1.0
50,beb87ea54499f95a614653924c99b9c4,514a4961e4b004a36b8d9ea0,Bar Isabel,"797 College St, Toronto ON M6G 1C7",414,-79.420682,43.654672,"[Cocktail Bar, American Restaurant, Tapas Rest...",9.1,3.0
405,cc1432c52b1e87a738180d369ed1136b,4fff1f96e4b042ae8acddca5,Fahrenheit Coffee,"120 Lombard St (at Jarvis St), Toronto ON M5C 3H5",628,-79.372916,43.652416,"[Café, Coffee Shop, Restaurant]",9.1,2.0
